# Advanced RVC Inference - No UI Version

This notebook provides a simplified, non-UI interface for RVC inference with command-line style operations.

In [ ]:
# Installation
import os
import sys
import subprocess
from pathlib import Path
import json

print("Setting up Advanced RVC environment...")

# Install dependencies
subprocess.run(["pip", "install", "-r", "requirements.txt"], check=True)

# Download prerequisites
subprocess.run(["python", "programs/applio_code/rvc/lib/tools/prerequisites_download.py"], check=True)

print("Setup completed successfully!")

In [ ]:
# Configuration parameters - edit these as needed

# Model settings
model_path = "models/your_model.pth"  # Path to your RVC model
index_path = "models/your_index.index"  # Path to your index file
model_name = "contentvec"  # or other model types

# Audio settings
input_audio_path = "audio_files/input.wav"  # Your input audio file
output_path = "audio_files/output.wav"  # Output file path

# Conversion parameters
pitch = 0  # Pitch adjustment
index_rate = 0.75
rms_mix_rate = 0.25
protect = 0.33
hop_length = 128
filter_radius = 3
split_audio = True
autotune = False
f0_method = "rmvpe"

# Processing options
vocal_model = "MDX23C"
karaoke_model = "UVR-BVE"
dereverb_model = "UVR-Deecho-Dereverb"
deecho = False
denoise = False
reverb = False

# Output format
export_format = "wav"
devices = "0" if __import__('torch').cuda.is_available() else "cpu"

print(f"Configuration set. Model: {model_path}, Input: {input_audio_path}")

In [ ]:
# Import required modules
from core import full_inference_program
import torch
import os

def check_files():
    """Check if required files exist"""
    missing = []
    if not os.path.exists(input_audio_path):
        missing.append(input_audio_path)
    if not os.path.exists(model_path):
        missing.append(model_path)
    if index_path and not os.path.exists(index_path):
        missing.append(index_path)
    
    if missing:
        print(f"Missing files: {missing}")
        return False
    return True

def run_conversion():
    """Run the RVC conversion with the specified parameters"""
    if not check_files():
        print("Cannot run conversion due to missing files")
        return None
    
    print("Starting RVC conversion...")
    try:
        result = full_inference_program(
            model_path=model_path,
            index_path=index_path,
            input_audio_path=input_audio_path,
            output_path=output_path,
            export_format_rvc=export_format,
            split_audio=split_audio,
            autotune=autotune,
            vocal_model=vocal_model,
            karaoke_model=karaoke_model,
            dereverb_model=dereverb_model,
            deecho=deecho,
            deecho_model="UVR-Deecho-Normal",
            denoise=denoise,
            denoise_model="UVR Denoise",
            reverb=reverb,
            vocals_volume=0.0,
            instrumentals_volume=0.0,
            backing_vocals_volume=0.0,
            export_format_final=export_format,
            devices=devices,
            pitch=pitch,
            filter_radius=filter_radius,
            index_rate=index_rate,
            rms_mix_rate=rms_mix_rate,
            protect=protect,
            pitch_extract=f0_method,
            hop_lenght=hop_length,
            reverb_room_size=0.15,
            reverb_damping=0.7,
            reverb_wet_gain=0.1,
            reverb_dry_gain=0.8,
            reverb_width=1.0,
            embedder_model=model_name,
            delete_audios=False,
            use_tta=False,
            batch_size=1,
            infer_backing_vocals=False,
            infer_backing_vocals_model="",
            infer_backing_vocals_index="",
            change_inst_pitch=0,
            pitch_back=0,
            filter_radius_back=3,
            index_rate_back=0.75,
            rms_mix_rate_back=0.25,
            protect_back=0.33,
            pitch_extract_back="harvest",
            hop_length_back=128,
            export_format_rvc_back=export_format,
            split_audio_back=False,
            autotune_back=False,
            embedder_model_back=model_name,
        )
        print(f"Conversion completed: {result[0]}")
        print(f"Output file: {result[1]}")
        return result[1]
    except Exception as e:
        print(f"Conversion failed: {e}")
        return None

In [ ]:
# Run the conversion
output_file = run_conversion()

if output_file:
    print(f"\n✅ Conversion successful! Output saved to: {output_file}")
    
    # Optional: Play the result
    try:
        from IPython.display import Audio, display
        display(Audio(output_file, autoplay=False))
        print("🎵 Audio preview loaded")
    except:
        print(f"Audio file created at: {output_file}")
else:
    print("\n❌ Conversion failed. Please check the configuration and try again.")

## Usage Instructions

1. Update the configuration parameters in the 'configuration' cell:
   - Set your model_path to the RVC model file
   - Set your input_audio_path to the audio file you want to convert
   - Adjust conversion parameters as needed

2. Run all cells in order

3. Check the output file after conversion completes

## Parameters Explanation

- `pitch`: Pitch adjustment in semitones
- `index_rate`: How much to use the index file (0.0-1.0)
- `f0_method`: Method for pitch extraction ('harvest', 'crepe', 'rmvpe', etc.)
- `split_audio`: Whether to split audio for processing large files
- `protect`: Protection for voiceless consonants (0.0-1.0)

## Note
This is a command-line style notebook with minimal UI elements. For more advanced features, use Advanced-RVC.ipynb